In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from torch import optim
from data_utils import *
from model_utils import *

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)

n_cudas = torch.cuda.device_count()
print(f"gpu count: {n_cudas}")
for i in range(n_cudas):
    print(torch.cuda.get_device_name(i))

gpu count: 0


# Prepare Dataset

In [3]:
# create raw dataset
x, y = GenerateRandomDataset.generate_slr_dataset_v2(
    true_bias=3.0, 
    true_weight=4.0, 
    sample_size=1000
)

In [4]:
# into Pytorch's tensor
x_tensor = torch.as_tensor(x).float().to(device)
y_tensor = torch.as_tensor(y).float().to(device)

In [5]:
# builds datasets
dataset = CustomDataset(x_tensor, y_tensor)

In [6]:
# performs the split
ratio = .8
n_total = len(dataset)
n_train = int(n_total * ratio)
n_val = n_total - n_train
train_data, val_data = random_split(dataset, [n_train, n_val])

In [7]:
# builds dataloaders
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=16, shuffle=True)

In [8]:
# retrieve mini-batches
# next(iter(train_loader))

# Define Model

In [9]:
# define model
class MyLinearRegressionNested(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(in_features=1, out_features=1)
    
    def forward(self, x):
        y_hat = self.linear(x)
        return y_hat

In [10]:
# create model instance
reg_model = MyLinearRegressionNested().to(device)

# Training Loop

## Mini-batch Gradient Descent

In [11]:
num_epochs = 20
lr = 0.1
optimizer = optim.SGD(params=reg_model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean')

perform_train_step = make_train_step_fn(model=reg_model, loss_func=loss_fn, optimizer=optimizer)
perform_val_step = make_val_step_fn(model=reg_model, loss_func=loss_fn)

In [12]:
losses = []
val_losses = []
for i in range(num_epochs):
    # train
    loss = mini_batch(device=device, data_loader=train_loader, step_fn=perform_train_step)
    losses.append(loss)
    
    # validation
    with torch.no_grad():
        val_loss = mini_batch(device=device, data_loader=val_loader, step_fn=perform_val_step)
        val_losses.append(val_loss)
    
    print(f"Epoch: {i}, Training Loss: {loss:0.5f}, Validation Loss: {val_loss:0.5f}")

Epoch: 0, Training Loss: 1.01664, Validation Loss: 0.07500
Epoch: 1, Training Loss: 0.04788, Validation Loss: 0.02577
Epoch: 2, Training Loss: 0.02023, Validation Loss: 0.01362
Epoch: 3, Training Loss: 0.01238, Validation Loss: 0.01178
Epoch: 4, Training Loss: 0.01023, Validation Loss: 0.01169
Epoch: 5, Training Loss: 0.00974, Validation Loss: 0.01519
Epoch: 6, Training Loss: 0.00971, Validation Loss: 0.01242
Epoch: 7, Training Loss: 0.00946, Validation Loss: 0.01353
Epoch: 8, Training Loss: 0.00963, Validation Loss: 0.01298
Epoch: 9, Training Loss: 0.00969, Validation Loss: 0.01198
Epoch: 10, Training Loss: 0.00963, Validation Loss: 0.01266
Epoch: 11, Training Loss: 0.00956, Validation Loss: 0.01218
Epoch: 12, Training Loss: 0.00956, Validation Loss: 0.01186
Epoch: 13, Training Loss: 0.00957, Validation Loss: 0.01249
Epoch: 14, Training Loss: 0.00965, Validation Loss: 0.01324
Epoch: 15, Training Loss: 0.00978, Validation Loss: 0.01201
Epoch: 16, Training Loss: 0.00965, Validation Loss

In [13]:
# Checks model's parameters
print(reg_model.state_dict())

OrderedDict({'linear.weight': tensor([[3.9904]]), 'linear.bias': tensor([2.9915])})
